In [29]:
import nibabel as nib
from nibabel.affines import apply_affine
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
dkatlas = nib.load('../Data/aparc+aseg.nii.gz')

In [27]:
hdr = dkatlas.header
affine = dkatlas.affine
data = np.asarray(dkatlas.dataobj)
atlasLUT = pd.read_csv('../Data/desikanKilliany.csv')

np.int32(10)

In [33]:
x,y,z = np.where(data == 10)
# convert to RAS coordinates
RAS = apply_affine(affine, np.array([x,y,z]).T)
RAS

In [30]:
apply_affine??

Signature: apply_affine(aff, pts, inplace=False)
Source:   
def apply_affine(aff, pts, inplace=False):
    """Apply affine matrix `aff` to points `pts`

    Returns result of application of `aff` to the *right* of `pts`.  The
    coordinate dimension of `pts` should be the last.

    For the 3D case, `aff` will be shape (4,4) and `pts` will have final axis
    length 3 - maybe it will just be N by 3. The return value is the
    transformed points, in this case::

        res = np.dot(aff[:3,:3], pts.T) + aff[:3,3:4]
        transformed_pts = res.T

    This routine is more general than 3D, in that `aff` can have any shape
    (N,N), and `pts` can have any shape, as long as the last dimension is for
    the coordinates, and is therefore length N-1.

    Parameters
    ----------
    aff : (N, N) array-like
        Homogeneous affine, for 3D points, will be 4 by 4. Contrary to first
        appearance, the affine will be applied on the left of `pts`.
    pts : (..., N-1) array-like
     